In [79]:
from rdkit import Chem
from rdkit.Chem import Descriptors
from rdkit.Chem.EnumerateStereoisomers import EnumerateStereoisomers, StereoEnumerationOptions

import numpy as np
from IPython.display import display
from rd import mol
from util import py3

BOND_ORDERS = {
    1: Chem.BondType.SINGLE,
    2: Chem.BondType.DOUBLE,
    3: Chem.BondType.TRIPLE,
}

def smiles_from_graphs(nodes, adjMtrx, newRIdx):
    mol = Chem.RWMol()
    atom_map = {}
    for i, atom_symbol in enumerate(nodes):
        atom = Chem.Atom(atom_symbol)
        idx = mol.AddAtom(atom)
        atom_map[i] = idx
    for i in range(len(nodes)):
        for j in range(i + 1, len(nodes)):
            if adjMtrx[i][j] != 0:
                mol.AddBond(atom_map[i], atom_map[j], BOND_ORDERS[adjMtrx[i][j]])
    mol.GetAtomWithIdx(newRIdx).SetNumRadicalElectrons(1)
    return Chem.MolToSmiles(mol.GetMol(), canonical=True)

def enumerate_stereoisomers(mol):
    opts = StereoEnumerationOptions(tryEmbedding=True, onlyUnassigned=False, unique=True)
    return tuple(EnumerateStereoisomers(mol, options=opts))

def canonicalize_transition(reSmi, prSmi):
    return tuple(sorted([reSmi, prSmi]))


def beta_cleavage(reSmi, adjMtrx, nodes, radicals, transitions, wells):
    for idxR in range(len(adjMtrx)):
        if radicals[idxR] == 0:
            continue

        for idxA, ordA in enumerate(adjMtrx[idxR]):
            if ordA <= 0:
                continue
            if nodes[idxA] == "H":
                continue

            for idxB, ordB in enumerate(adjMtrx[idxA]):
                if ordB != 1:
                    continue
                if idxB == idxR:
                    continue
                if nodes[idxB] == "H":
                    continue

                trAdjMtrx = np.triu(adjMtrx.copy())
                i1, j1 = sorted((idxA, idxR))
                trAdjMtrx[j1, i1] += 1

                i2, j2 = sorted((idxA, idxB))
                trAdjMtrx[j2, i2] -= 1

                prAdjMtrx = trAdjMtrx + trAdjMtrx.T
                prAdjMtrx = np.where(prAdjMtrx, prAdjMtrx, prAdjMtrx.T)

                prSmi = smiles_from_graphs(nodes, prAdjMtrx, idxB)

                canonTr = canonicalize_transition(reSmi, prSmi)

                if canonTr not in transitions:
                    transitions[canonTr] = trAdjMtrx
                    prMol = mol.from_smiles(prSmi)
                    frMols = Chem.GetMolFrags(prMol, asMols=True)
                    for frMol in frMols:
                        nRad = Descriptors.NumRadicalElectrons(frMol)
                        if nRad > 0:
                            frSmi = Chem.MolToSmiles(frMol)
                            if frSmi in wells:
                                continue
                            explore_products(frSmi, wells, transitions)
                        else:
                            sterMols = enumerate_stereoisomers(frMol)
                            for sterMol in sterMols:
                                sterSmi = Chem.MolToSmiles(sterMol, isomericSmiles=True)
                                if sterSmi in wells:
                                    continue
                                wells[sterSmi] = {
                                    "charge": Chem.GetFormalCharge(frMol),
                                    "multiplicity": nRad + 1,
                                    "nodes": nodes.copy(),
                                    "adjacency": prAdjMtrx.copy()
                                }


def proton_transfer(reSmi, adjMtrx, nodes, radicals, transitions, wells):
    for idxR in range(len(adjMtrx)):
        if radicals[idxR] == 0:
            continue
        for idxH in range(len(adjMtrx)):
            if nodes[idxH] == "H":
                idxD = np.where(adjMtrx[idxH] == 1)[0].item()
                if not idxD == idxR:
                    trAdjMtrx = np.triu(adjMtrx.copy())

                    i1, j1 = sorted((idxH, idxR))
                    trAdjMtrx[j1, i1] += 1
                    i2, j2 = sorted((idxH, idxD))
                    trAdjMtrx[j2, i2] += -1

                    prAdjMtrx = trAdjMtrx + trAdjMtrx.T
                    prAdjMtrx = np.where(prAdjMtrx, prAdjMtrx, prAdjMtrx.T)

                    prSmi = smiles_from_graphs(nodes, prAdjMtrx, idxD)

                    canonTr = canonicalize_transition(reSmi, prSmi)

                    if canonTr not in transitions:
                        transitions[canonTr] = trAdjMtrx
                        explore_products(prSmi, wells, transitions)


def explore_products(smiles: str, wells: dict, transitions: dict):
    reMol = mol.from_smiles(smiles, with_coords=True)
    reSmi = Chem.MolToSmiles(reMol, canonical=True)

    if reSmi in wells:
        return

    adjMtrx = Chem.rdmolops.GetAdjacencyMatrix(reMol, useBO=True)
    nodes = [reMol.GetAtomWithIdx(i).GetSymbol() for i in range(len(adjMtrx))]
    radicals = [reMol.GetAtomWithIdx(i).GetNumRadicalElectrons() for i in range(len(adjMtrx))]
    wells[reSmi] = {
        "charge": Chem.GetFormalCharge(reMol),
        "multiplicity": sum(radicals) + 1,
        "nodes": nodes,
        "adjacency": adjMtrx.copy()
    }

    proton_transfer(reSmi, adjMtrx, nodes, radicals, transitions, wells)

    beta_cleavage(reSmi, adjMtrx, nodes, radicals, transitions, wells)


# --- Run it ---
smiles = "[CH2]CCC1CO1"
wells = {}
transitions = {}

explore_products(smiles, wells, transitions)

# --- Display results ---
print(f"\n{len(wells)} Molecules Discovered:")
for smi in wells:
    print(smi)

print(f"\n{len(transitions)} Transitions Discovered:")
for key, mat in transitions.items():
    print(f"{key}")
    print(mat)


104 Molecules Discovered:
[H][C]([H])C([H])([H])C([H])([H])C1([H])OC1([H])[H]
[H][C](C([H])([H])[H])C([H])([H])C1([H])OC1([H])[H]
[H][C](C([H])([H])C([H])([H])[H])C1([H])OC1([H])[H]
[H]C1([H])O[C]1C([H])([H])C([H])([H])C([H])([H])[H]
[H][C]1OC1([H])C([H])([H])C([H])([H])C([H])([H])[H]
[H][C](C([H])=O)C([H])([H])C([H])([H])C([H])([H])[H]
[H]C([H])([H])C([H])([H])C([H])([H])C([H])([H])[C]=O
[H][C]([H])C([H])([H])C([H])([H])C([H])([H])C([H])=O
[H][C](C([H])([H])[H])C([H])([H])C([H])([H])C([H])=O
[H][C](C([H])([H])C([H])=O)C([H])([H])C([H])([H])[H]
[H]C([H])=C([H])C([H])([H])C([H])([H])[H]
[H][C]=O
[H]C(=O)C([H])([H])C([H])=C([H])[H]
[H][C]([H])[H]
[H]C([H])=C([H])C([H])([H])[H]
[H][C]([H])C([H])=O
[H]C([H])([H])[C]=O
[H]C([H])=C([H])[H]
[H][C]([H])C([H])([H])C([H])=O
[H][C](C([H])=O)C([H])([H])[H]
[H]C([H])([H])C([H])([H])[C]=O
[H]C([H])=C=O
[H][C]([H])C([H])([H])C([H])([H])[H]
[H][C](C([H])([H])[H])C([H])([H])[H]
[H]C(=O)C([H])=C([H])[H]
[H][C]([H])C([H])([H])[H]
[H]C([O])=C([H])C([H])(